In [ ]:
from odps import ODPS
from odps import options
from odps.df import DataFrame
import pandas as pd
import numpy as np

ACCESS_ID = 'LTAI5t9uwJrh5eJ7Q5E37D1s'
SECRET_ACCESS_KEY = 'NCFHOAnvqfnTrpypgR4b3cNawP8fnB'
ODPS_PROJECT = 'OpenDigger_prod_dev'
ODPS_ENDPOINT = 'http://service.cn-shanghai.maxcompute.aliyun.com/api'
# 创建odps实例
o = ODPS(ACCESS_ID, SECRET_ACCESS_KEY,
         project=ODPS_PROJECT, endpoint=ODPS_ENDPOINT)
options.tunnel.limit_instance_tunnel = False
# options.read_timeout = 3600000

In [ ]:
# 创建临时数据库
TEMP_TABLE_NAME = 'test_repo'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        actor_id BIGINT,
        type String,
        action String,
        actor_login String,
        created_date DATETIME,
        issue_number INT,
        issue_comments SMALLINT,
        pull_commits SMALLINT,
        pull_merged TINYINT,
        pull_review_comments SMALLINT,
        repo_forks_count INT
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

In [ ]:
select_sql = '''
INSERT INTO {TEMP_TABLE_NAME}   
SELECT
    actor_id,
    type,
    action,
    actor_login,
    created_at,
    issue_number,
    issue_comments,
    pull_commits,
    pull_merged,
    pull_review_comments,
    repo_forks_count
FROM ods_github_log
WHERE repo_name='PaddlePaddle/Paddle'
and pt>'20210101' and pt <= '20211201'
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME
)

table = o.execute_sql(select_sql ,hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

In [ ]:
# 获取需要导出的项目
select_sql = '''
    set odps.sql.validate.orderby.limit=false;
    SELECT  
        actor_id,
        actor_login,
        type,
        action,
        created_date,
        issue_number,
        issue_comments,
        pull_commits,
        pull_merged,
        pull_review_comments,
        repo_forks_count
    FROM    {TEMP_TABLE_NAME}
    '''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME,
)
table = o.execute_sql(select_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

#保存本地
data_test=pd.DataFrame(columns=['actor_id','actor_login','type','action','created_date','issue_number','issue_comments','pull_commits','pull_merged','pull_review_comments','repo_forks_count'])
i = 0
with table.open_reader() as reader:
    for record in reader:
#         if i==10:
#             break
        data_test.loc[i]=[record[0],record[1],record[2],record[3],record[4],record[5],record[6],record[7],record[8],record[9],record[10]]
        i+=1   


In [ ]:
data_test[data_test['actor_login']!='paddle-bot[bot]']

In [ ]:
data_test.to_csv('Paddle_2021_loc.csv')

In [ ]:
data=pd.read_csv('Paddle_2021_loc.csv')
data

In [ ]:
data=data[data['actor_login']!='paddle-bot[bot]']
data

In [ ]:
data=data.fillna(0)
data

In [ ]:
def categorize(row):
    return 1 if row['pull_merged']!=0 else 0

In [ ]:
data['contributer']=data.apply(lambda row:categorize(row),axis=1)
data

In [ ]:
data[data['contributer']!=0].shape

In [ ]:
import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
X=data.drop(columns=['Unnamed: 0','actor_id','actor_login','type','action','created_date','contributer'])
y=data['contributer']

In [ ]:
model=xgb.XGBClassifier(max_depth=10, gamma=0.5, learning_rate= 0.1)
model.fit(X,y)

In [ ]:
fig, ax1 = plt.subplots(figsize=(8,15))
xgb.plot_importance(model, ax=ax1)